# MIMIC IV ED Data
We use data from the MIMIC IV ED dataset. In particular, we use the following tables:
- `triage.csv` - contains triage information for each patient visit, the `chief_complaint` column is the free-text column of interest
- `pyxis.csv` - contains medication information for each patient visit, we filter `gsn` column for antibiotics

## Load libraries and data

In [1]:
# Load libraries
import os
import pandas as pd

from pathlib import Path

# Print current working directory
print(os.getcwd())

/home/kevin/DPhil/Projects/EHR-Indication-Processing/01_Preprocessing


In [2]:
# Load the data
data_dir = Path("../00_Data/")
mimic_ed_dir = data_dir / "MIMIC-IV-ED/"
lut_dir = data_dir / "LUTs/"

# MIMIC IV ED data
triage = pd.read_csv(mimic_ed_dir / "triage.csv")
pyxis = pd.read_csv(mimic_ed_dir / "pyxis.csv")

# LUTs to filter the medication for ABX
nddf_to_rxcui = pd.read_csv(lut_dir / "nddf_to_rxcui.csv")\
    .astype({"NDDF": int})
rxcui_abx_list = pd.read_csv(lut_dir / "abx_rxcui.csv")

## Data pre-processing & exploratory data analysis

Match the antibiotics with RxNorm IDs to NDDF (GSN) indentifiers

In [3]:
rxcui_abx_nddf = rxcui_abx_list.merge(nddf_to_rxcui, how="left", on="RXCUI")
rxcui_abx_nddf

,RXCUI,NDDF
0,211,NaN
1,239,NaN
2,268,NaN
3,270,NaN
4,388,NaN
...,...,...
4983,2198370,80112.0
4984,2265706,80508.0
4985,2265711,NaN
4986,2375333,81111.0


Create an overview of the joined table

In [4]:
abx_no_nddf = rxcui_abx_nddf.query(
    "NDDF.isnull()"
)  # antibiotics do not have a NDDF ID

print(f"Number of antibiotics without NDDF ID: {abx_no_nddf.shape[0]}")
print(f"Percentage of antibiotics without NDDF ID: {abx_no_nddf.shape[0]/rxcui_abx_nddf.shape[0]}")

Number of antibiotics without NDDF ID: 3916
Percentage of antibiotics without NDDF ID: 0.785084202085004


Check the MIMIC dataset for missing NDDF (GSN) IDs

In [5]:
# Check the MIMIC dataset for missing NDDF (GSN) IDs
pyxis_no_nddf = pyxis.query("gsn.isnull()")

# Summarise the medication name with missing GSN IDs, sort by frequency
medication_no_nddf = pyxis_no_nddf.groupby("name").size().sort_values(ascending=False)

print(f"Number of pyxis entries without NDDF ID: {pyxis_no_nddf.shape[0]}")
print(f"Percentage of pyxis entries without NDDF ID: {pyxis_no_nddf.shape[0]/pyxis.shape[0]}")
print(medication_no_nddf)

Number of pyxis entries without NDDF ID: 35452
Percentage of pyxis entries without NDDF ID: 0.02235234257619386
name
CefTRIAXone 1gm/100mL 100mL Bag           4312
CefePIME                                  3508
Ampicillin-Sulbactam                      3384
Readi-Cat 2  (Barium Sul 2%)*             3207
CeftriaXONE (Mini Bag Plus)               2863
                                          ... 
Morphine Sulfate (Pharmacy Compounded)       2
Cyano KIT                                    1
LACOsamide 100mg/100mL 100mL Bag             1
LACOsamide 150mg/100mL 100mL Bag             1
Ledipasvir/Sofosbuvir                        1
Length: 62, dtype: int64


-> Requires cleanup...

## Filter for visists with antibiotics

Filter for antibiotics in the prescription system

In [6]:
# Deduplicate the source datasets before joining
nddf_abx = rxcui_abx_nddf[~rxcui_abx_nddf.NDDF.isna()].drop_duplicates(subset="NDDF")
pyxis_nafree = pyxis[~pyxis.gsn.isna()]

pysis_abx = pyxis_nafree.merge(
    nddf_abx, how="inner", left_on="gsn", right_on="NDDF", validate="many_to_one"
)
pysis_abx

,subject_id,stay_id,charttime,med_rn,name,gsn_rn,gsn,RXCUI,NDDF
0,10000108,39513268,2163-09-24 20:45:00,1,Penicillin V Potassium,1,8879.0,834061,8879.0
1,10000108,39513268,2163-09-24 20:46:00,2,Penicillin V Potassium,1,8879.0,834061,8879.0
2,10021784,30314424,2189-03-11 00:32:00,2,Penicillin V Potassium,1,8879.0,834061,8879.0
3,10021784,30314424,2189-03-11 00:37:00,3,Penicillin V Potassium,1,8879.0,834061,8879.0
4,10027100,33852641,2155-09-28 20:50:00,1,Penicillin V Potassium,1,8879.0,834061,8879.0
...,...,...,...,...,...,...,...,...,...
131964,18001762,32047990,2160-10-11 04:41:00,8,Cortisporin Otic Susp,1,48559.0,310687,48559.0
131965,18567594,35615182,2206-03-05 23:20:00,2,Cortisporin Otic Susp 10mL DBTL,1,48559.0,310687,48559.0
131966,18724442,30826477,2152-10-22 21:17:00,1,Cortisporin Otic Susp,1,48559.0,310687,48559.0
131967,18852043,31889721,2185-10-15 11:20:00,4,Cortisporin Otic Susp,1,48559.0,310687,48559.0


Get `subject_id` and `stay_id`s with prescribed antibiotics

In [7]:
pysis_abx_patient = pysis_abx.drop_duplicates(subset=["subject_id", "stay_id"])
triage_abx = triage.merge(pysis_abx_patient, how="inner", on=["subject_id", "stay_id"])

In [8]:
complaints_table = triage_abx.chiefcomplaint

## Export the data

In [9]:
# Output path
mimic_complaint_path = mimic_ed_dir / "mimic_abx_complaint.csv"

In [ ]:
# Write the data
complaints_table.to_csv(mimic_complaint_path, index=False)